# Linear Algebra with Scipy


*Reading time ~30 min*

- *Author*: Vincent Maladiere
- *Credits*: [Scipy linalg Tutorial](https://docs.scipy.org/doc/scipy/tutorial/linalg.html)


`scipy.linalg` provide efficient linear algebra operation through BLAS and LAPACK backend. Input and ouput always expect 2-D arrays.

In [1]:
from scipy import linalg
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

## Standard operations

### Finding the determinant

The determinant of a square matrix $A$ is often denoted $|A|$ and is a quantity often used in linear algebra.

2 computation techniques:

**1. Leibniz formula**

$$A=\begin{bmatrix} a_{1,1} & a_{1,2} & a_{1,3} \\ a_{2,1} & a_{2,2} & a_{2,3} \\ a_{3,1} & a_{3,2} & a_{3, 3}\end{bmatrix}$$

<br>

| Permutation $\sigma$ | ${sgn}(\sigma )$ | $sgn(\sigma) \prod _{i=1}^{n}a_{i,\sigma _{i}}$ |
|--------------------------------------|-----------------------------------------------|--------------------------------------------------------------------------------|
| 1, 2, 3                              | $+1$                           | $+a_{1,1}a_{2,2}a_{3,3}$                                        |
| 1, 3, 2                              | $-1$                           | $-a_{1,1}a_{2,3}a_{3,2}$                                        |
| 3, 1, 2                              | $+1$                           | $+a_{1,3}a_{2,1}a_{3,2}$                                        |
| 3, 2, 1                              | $-1$                           | $-a_{1,3}a_{2,2}a_{3,1}$                                        |
| 2, 3, 1                              | $+1$                           | $+a_{1,2}a_{2,3}a_{3,1}$                                        |
| 2, 1, 3                              | $-1$                           | ${-a_{1,2}a_{2,1}a_{3,3}}$                                        |


The sign of $\sigma$  is defined to be $+1$ whenever the reordering given by $\sigma$ can be achieved by successively interchanging two entries an even number of times, and $-1$ whenever it can be achieved by an odd number of such interchanges.


$$det(A) = \sum_{\sigma \in S_n} \Big( sgn(\sigma) \prod^n_{i=1} a_{i,\sigma_i} \Big) = a_{1,1}a_{2,2}a_{3,3} - a_{1,1}a_{2,3}a_{3,2} + a_{1,3}a_{2,1}a_{3,2} - a_{1,3}a_{2,2}a_{3,1} + a_{1,2}a_{2,3}a_{3,1} - a_{1,2}a_{2,1}a_{3,3}$$

**2. Laplace expansion**

![laplace-expansion](../asset/laplaceexpansion.png)

Suppose  $a_{ij}$ are the elements of the matrix $A$ and let $M_{ij}=|A_{ij}|$ be the determinant of the matrix left by removing the $i$ row and $j$ column from $A$. Then, for any column $j$:

$$det(A)=\sum_{i=1}^N (-1)^{i+j} a_{ij} M_{ij}$$

This is a recursive way to define the determinant, where the base case is defined by accepting that the determinant of a $1 \times 1$ matrix is the only matrix element.

In Scipy:

In [32]:
A = np.array([[1, 3, 5], [2, 5, 1], [2, 3, 8]])
print(A, "\n")
print("det:", linalg.det(A))

[[1 3 5]
 [2 5 1]
 [2 3 8]] 

det: -25.000000000000004


In [31]:
A_singular = np.array([[1, 3, 5], [2, 6, 10], [2, 3, 8]])
print(A_singular, "\n")
print("det:", linalg.det(A_singular))

[[ 1  3  5]
 [ 2  6 10]
 [ 2  3  8]] 

det: 1.77635683940025e-15


### Finding the inverse

The inverse of a matrix $A$ is the matrix $B$, such that $AB = I$, where $I$ is the identity matrix consisting of ones down the main diagonal. Usually, $B$ is denoted  $B=A^{-1}$.

In SciPy, the matrix inverse of the NumPy array, A, is obtained using `linalg.inv(A)`, or using `A.I` if `A` is a Matrix. 

Under the hood, it performs [Gaussian elimination](https://en.wikipedia.org/wiki/Invertible_matrix#Gaussian_elimination), via [Lower-Upper decomposition](https://en.wikipedia.org/wiki/LU_decomposition).

For example:

$$A=\begin{bmatrix} 1 & 3 & 5 \\ 2 & 5 & 1 \\ 2 & 3 & 8 \end{bmatrix}$$

In [94]:
A = np.array([[1, 3, 5], [2, 5, 1], [2, 3, 8]])
A_inv = linalg.inv(A)
print(A, "\n")
print(A_inv, "\n")
print(A.dot(A_inv))

[[1 3 5]
 [2 5 1]
 [2 3 8]] 

[[-1.48  0.36  0.88]
 [ 0.56  0.08 -0.36]
 [ 0.16 -0.12  0.04]] 

[[ 1.00000000e+00 -1.11022302e-16  4.85722573e-17]
 [ 3.05311332e-16  1.00000000e+00  7.63278329e-17]
 [ 2.22044605e-16 -1.11022302e-16  1.00000000e+00]]


Dot product alternatives:

In [34]:
print(np.dot(A, A_inv), "\n")
print(A @ A_inv)

[[ 1.00000000e+00 -1.11022302e-16  4.85722573e-17]
 [ 3.05311332e-16  1.00000000e+00  7.63278329e-17]
 [ 2.22044605e-16 -1.11022302e-16  1.00000000e+00]] 

[[ 1.00000000e+00 -1.11022302e-16  4.85722573e-17]
 [ 3.05311332e-16  1.00000000e+00  7.63278329e-17]
 [ 2.22044605e-16 -1.11022302e-16  1.00000000e+00]]


**TODO #1**: Try to perform inversion on `A_singular`. What result do you expect?

In [ ]:
### TODO - write your code below ###


### Solving a linear system

Suppose we have the following equations to solve:
    
$$x + 3y + 5z = 10 \\ 2x + 5y + z = 8 \\ 2x + 3y + 8z = 3$$

We could turn this problem into matrix computation:

$$\begin{bmatrix} 1 & 3 & 5 \\ 2 & 5 & 1 \\ 2 & 3 & 8 \end{bmatrix} \begin{bmatrix}x\\y\\z\end{bmatrix}=\begin{bmatrix}10\\8\\3\end{bmatrix}$$

Thus our problem is now:

$$\begin{bmatrix}x\\y\\z\end{bmatrix} = \begin{bmatrix} 1 & 3 & 5 \\ 2 & 5 & 1 \\ 2 & 3 & 8 \end{bmatrix}^{-1} \begin{bmatrix}10\\8\\3\end{bmatrix} $$

**TODO #2**: Solve the problem using only `linalg.inv`

In [92]:
A = np.array([[1, 3, 5], [2, 5, 1], [2, 3, 8]])
b = np.array([[10], [8], [3]])

### TODO - write your code below ###


However, it is better to use the `linalg.solve` command, which can be faster and more numerically stable. In this case, it, however, gives the same answer as shown in the following example:

In [37]:
x = linalg.solve(A, b)
x

array([[-9.28],
       [ 5.16],
       [ 0.76]])

Check:

In [38]:
A @ x - b

array([[ 0.00000000e+00],
       [-1.77635684e-15],
       [-1.77635684e-15]])

### Norms

A wide range of norm definitions are available using different parameters to the order argument of `linalg.norm`. This function takes a rank-1 (vectors) or a rank-2 (matrices) array and an optional order argument (default is 2). Based on these inputs, a vector or matrix norm of the requested order is computed.

For a vector

$$||x||=\begin{cases} max|x_i| & ord=inf\\ min|x_i| & ord=-inf \\ \big(\sum_i |x_i|^{ord} \big)^{1/ord} & ord < inf \end{cases}$$

For a matrix

$$||A||=\begin{cases} max_i \sum_j |a_{ij}| & ord = inf \\ min_i \sum_j |a_{ij}| & ord = -inf \\ max_j \sum_i |a_{ij}| & ord = 1\\ min_j \sum_i |a_{ij}| & ord = -1 \\ max\, \sigma_i & ord = 2 \\ max \,\sigma_j & ord = -2 \\ \sqrt{\mathrm{trace}(A^TA}) & ord = \mathrm{"fro"}\end{cases}$$

In [99]:
A

array([[1, 3, 5],
       [2, 5, 1],
       [2, 3, 8]])

In [100]:
A[0]

array([1, 3, 5])

In [103]:
print(linalg.norm(A[0], np.inf))
print(linalg.norm(A[0], -np.inf))
print(linalg.norm(A[0], 2))

5.0
1.0
5.916079783099616


In [106]:
print(linalg.norm(A))  # frobenius "fro" is the default
print(linalg.norm(A, np.inf))
print(linalg.norm(A, 1))
print(linalg.norm(A, 2))

11.916375287812984
13.0
14.0
11.133851342531154


### Eigenvalues and engenvectors

The eigenvalue-eigenvector problem is one of the most commonly employed linear algebra operations. In one popular form, the eigenvalue-eigenvector problem is to find for some square matrix $A$, scalars $\lambda$ and corresponding vectors $v$, such that:

$$A v = \lambda v$$

For an $N \times N$ matrix, there are  (not necessarily distinct) eigenvalues — roots of the (characteristic) polynomial

$$| A - \lambda I | = 0$$

Let's consider again the matrix:

$$A=\begin{bmatrix} 1 & 5 & 2 \\ 2 & 4 & 1 \\ 3 & 6 & 2 \end{bmatrix}$$

The characteristic polynomial is:

$$|A - \lambda I| = \begin{bmatrix} 1-\lambda & 5 & 2 \\ 2 & 4-\lambda & 1 \\ 3 & 6 & 2-\lambda \end{bmatrix} = (1-\lambda) [(4-\lambda)(2-\lambda)-6]-5[2(2-\lambda)-3]+2[12-3(4-\lambda)]\\=-\lambda^3+7\lambda^2+8\lambda-3$$ 

The roots of this polynomial are:

$$\lambda_1 = 7.9579\\\lambda_2 = -1.2577\\\lambda_3=0.2997$$

In [65]:
A = np.array([[1, 5, 2], [2, 4, 1], [3, 6, 2]])
la, v = linalg.eig(A)
print("lambdas:\n", la, "\n")
print("right v:\n", v)

lambdas:
 [ 7.9579162 +0.j -1.25766471+0.j  0.2997485 +0.j] 

right v:
 [[-0.5297175  -0.90730751  0.28380519]
 [-0.44941741  0.28662547 -0.39012063]
 [-0.71932146  0.30763439  0.87593408]]


Sanity check the initial equation $A v_i = \lambda_i v_i $

In [88]:
for idx in range(3):
    print(linalg.norm(A @ v[:, idx] - la[idx] * v[:, idx]))

3.233018248352212e-15
1.1389935954777104e-15
4.871083751574258e-16


In [66]:
A @ v

array([[-4.21544748,  1.14108863,  0.08507018],
       [-3.57642611, -0.36047874, -0.11693807],
       [-5.72429989, -0.38690092,  0.26255993]])

In [67]:
la * v

array([[-4.21544748+0.j,  1.14108863-0.j,  0.08507018+0.j],
       [-3.57642611+0.j, -0.36047874+0.j, -0.11693807+0.j],
       [-5.72429989+0.j, -0.38690092+0.j,  0.26255993+0.j]])

In [71]:
A @ v - la * v

array([[-2.66453526e-15+0.j,  0.00000000e+00+0.j,  5.55111512e-17+0.j],
       [-4.44089210e-16+0.j,  8.32667268e-16+0.j,  1.11022302e-16+0.j],
       [-1.77635684e-15+0.j,  7.77156117e-16+0.j,  2.22044605e-16+0.j]])

**TODO #3**: What operation does `la * v` realize?

Eigenvectors are unitary (defining a orthonormal basis)

In [79]:
(v**2)

array([[0.28060063, 0.82320692, 0.08054539],
       [0.20197601, 0.08215416, 0.15219411],
       [0.51742336, 0.09463892, 0.76726051]])

In [80]:
(v**2).sum(axis=0)

array([1., 1., 1.])

### Singular value decomposition (SVD)

Singular value decomposition (SVD) can be thought of as an extension of the eigenvalue problem to matrices that are not square. 

Let $A$ be a $N \times M$ matrix. $A^T A$ and $A A ^T$ are squared hermitian matrix of dimension $M \times M$ and $N \times N$. They are real, positive-definite matrix and hence invertible. 

Let's call their eigenvalues $\sigma_i^2$. The singular values of $A$ are then $\sigma_i$.

$A$ also admit the SVD decomposition:

$$A = U \Sigma V^T $$

With 
- $\Sigma = diag(\sigma_1, \sigma_2, ..., \sigma_k)$
- $k = min(M, N)$.

In [124]:
A = np.array([[1, 2, 3], [4, 5, 6]])
n, m = A.shape
u, s, v = linalg.svd(A)
print(u, "\n")
print(s, "\n")
print(v, "\n")

[[-0.3863177  -0.92236578]
 [-0.92236578  0.3863177 ]] 

[9.508032   0.77286964] 

[[-0.42866713 -0.56630692 -0.7039467 ]
 [ 0.80596391  0.11238241 -0.58119908]
 [ 0.40824829 -0.81649658  0.40824829]] 



In [126]:
S = linalg.diagsvd(s, n, m)
S

array([[9.508032  , 0.        , 0.        ],
       [0.        , 0.77286964, 0.        ]])

**TODO #4** Check that the decomposition is correct by computing the Frobenius norm.

In [129]:
### TODO - Write your code below ### 



## Exercice